#Libraries

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
import pandas as pd
import numpy as np
import re
import nltk

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import ast##Abstract Syntax Tree..one function in python

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

#Looking at our dataset and creating a overall dataset we need

In [53]:
movie_content=pd.read_csv("/content/drive/MyDrive/datasets/Dataset/movies_metadata.csv")

<ipython-input-53-76982377f6c9>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movie_content=pd.read_csv("/content/drive/MyDrive/datasets/Dataset/movies_metadata.csv")


In [10]:
movie_content.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
movie_content.shape

(45466, 24)

In [12]:
movie_content.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [16]:
movie_content[movie_content["adult"]!="False"]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19489,True,NaN,0,"[{'id': 27, 'name': 'Horror'}]",NaN,5422,tt0079642,it,Le notti erotiche dei morti viventi,A sailor takes an American businessman and his...,...,1980-11-18,0.0,112.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}]",Released,NaN,Erotic Nights of the Living Dead,False,2.2,7.0
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28701,True,NaN,0,"[{'id': 80, 'name': 'Crime'}]",http://www.imdb.com/title/tt0119931/,169158,tt0119931,en,Standoff,The FBI and Texas police make ready to storm t...,...,1998-01-11,0.0,91.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Trust Will Get You Killed,Standoff,False,4.8,5.0
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31934,True,NaN,0,"[{'id': 35, 'name': 'Comedy'}]",NaN,44781,tt0322232,cn,發電悄嬌娃,Electrical Girl centers around a horny young w...,...,2001-04-26,0.0,89.0,"[{'iso_639_1': 'cn', 'name': '广州话 / 廣州話'}]",Released,NaN,Electrical Girl,False,0.0,0.0
32113,True,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",http://www.dietofsex.com/,324230,tt3094816,es,Diet of Sex,Ágata suffers from a psychological disorder wh...,...,2014-02-14,0.0,72.0,"[{'iso_639_1': 'es', 'name': 'Español'}]",Released,"Comedy, food, drama and sex, a lot of sex",Diet of Sex,False,4.0,12.0
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39901,True,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 27, 'name...",NaN,35731,tt1161951,en,Amateur Porn Star Killer 2,Shane Ryan's sequel to the disturbing Amateur ...,...,2008-05-13,0.0,0.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Humiliation. Rape. Murder. You know the drill.,Amateur Porn Star Killer 2,False,6.3,8.0
39902,True,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10402, '...",NaN,55774,tt1153101,en,The Band,Australian film about a fictional sub-par Aust...,...,2009-11-17,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,"Sex, drugs and Rock 'n Roll",The Band,False,3.3,7.0
40574,True,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 27, 'name...",NaN,39922,tt0070696,da,Dværgen,Olaf and his mother run a boarding house and a...,...,1973-12-20,0.0,92.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Mother of all Dwarfsploitation films!,The Sinful Dwarf,False,5.8,13.0


In [14]:
movie_content.iloc[[19730,29503,35587]]## these rows have adult content. so we are dropping them

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
movie_content = movie_content.drop([19730, 29503, 35587])

In [18]:
movie_content.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [19]:
movie_content.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45463 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45463 non-null  object 
 1   belongs_to_collection  4491 non-null   object 
 2   budget                 45463 non-null  object 
 3   genres                 45463 non-null  object 
 4   homepage               7779 non-null   object 
 5   id                     45463 non-null  object 
 6   imdb_id                45446 non-null  object 
 7   original_language      45452 non-null  object 
 8   original_title         45463 non-null  object 
 9   overview               44509 non-null  object 
 10  popularity             45460 non-null  object 
 11  poster_path            45077 non-null  object 
 12  production_companies   45460 non-null  object 
 13  production_countries   45460 non-null  object 
 14  release_date           45376 non-null  object 
 15  re

In [55]:
movie_content['id'] = movie_content['id'].astype(int)

In [56]:
links_small = pd.read_csv('/content/drive/MyDrive/datasets/Dataset/links_small.csv')

In [22]:
links_small.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [23]:
links_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  9125 non-null   int64  
 1   imdbId   9125 non-null   int64  
 2   tmdbId   9112 non-null   float64
dtypes: float64(1), int64(2)
memory usage: 214.0 KB


In [ ]:
links_small.shape

(9125, 3)

In [57]:
links_small = links_small[links_small['tmdbId'].notnull()]['tmdbId'].astype('int')
##tmbId is equal to movieId.. so we extract that only

In [58]:
links_small

0          862
1         8844
2        15602
3        31357
4        11862
         ...  
9120    402672
9121    315011
9122    391698
9123    137608
9124    410803
Name: tmdbId, Length: 9112, dtype: int64

In [59]:
movie_content = movie_content[movie_content['id'].isin(links_small)]

In [60]:
len(movie_content)

9099

Now we need add  cast and crew data to our movies


In [61]:
credits_data = pd.read_csv('/content/drive/MyDrive/datasets/Dataset/credits.csv')

In [28]:
credits_data.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [29]:
credits_data['cast'][0]

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [62]:
def convert(text):
    L = []
    for i in ast.literal_eval(text):##to iterates thru each dictionary
      L.append(i['name']) 
    return L 

#ast.literal_eval() --commonly used to safely evaluate and parse a string representation of a
 #Python literal, such as a list, dictionary, number, or string, into its corresponding Python object.

In [63]:
credits_movies = pd.DataFrame()

In [64]:
cast_list = []
for ind in credits_data.index:##to iterates over each row
  cast_list.append(convert(credits_data['cast'][ind]))

In [65]:
cast_list[0]

['Tom Hanks',
 'Tim Allen',
 'Don Rickles',
 'Jim Varney',
 'Wallace Shawn',
 'John Ratzenberger',
 'Annie Potts',
 'John Morris',
 'Erik von Detten',
 'Laurie Metcalf',
 'R. Lee Ermey',
 'Sarah Freeman',
 'Penn Jillette']

In [66]:
crew_list = []
for ind in credits_data.index:
  crew_list.append(convert(credits_data['crew'][ind]))

In [35]:
crew_list[0]

['John Lasseter',
 'Joss Whedon',
 'Andrew Stanton',
 'Joel Cohen',
 'Alec Sokolow',
 'Bonnie Arnold',
 'Ed Catmull',
 'Ralph Guggenheim',
 'Steve Jobs',
 'Lee Unkrich',
 'Ralph Eggleston',
 'Robert Gordon',
 'Mary Helen Leasman',
 'Kim Blanchette',
 'Marilyn McCoppen',
 'Randy Newman',
 'Dale E. Grahn',
 'Robin Cooper',
 'John Lasseter',
 'Pete Docter',
 'Joe Ranft',
 'Patsy Bouge',
 'Norm DeCarlo',
 'Ash Brannon',
 'Randy Newman',
 'Roman Figun',
 'Don Davis',
 'James Flamberg',
 'Mary Beth Smith',
 'Rick Mackay',
 'Susan Bradley',
 'William Reeves',
 'Randy Newman',
 'Andrew Stanton',
 'Pete Docter',
 'Gary Rydstrom',
 'Karen Robert Jackson',
 'Chris Montan',
 'Rich Quade',
 'Michael Berenstein',
 'Colin Brady',
 'Davey Crockett Feiten',
 'Angie Glocka',
 'Rex Grignon',
 'Tom K. Gurney',
 'Jimmy Hayward',
 'Hal T. Hickel',
 'Karen Kiser',
 'Anthony B. LaMolinara',
 'Guionne Leroy',
 'Bud Luckey',
 'Les Major',
 'Glenn McQueen',
 'Mark Oftedal',
 'Jeff Pidgeon',
 'Jeff Pratt',
 'Stev

In [67]:
credits_movies['id'] = credits_data['id']

In [68]:
credits_movies['cast'] = cast_list

In [69]:
credits_movies['crew'] = crew_list

In [70]:
credits_movies

,id,cast,crew
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J..."
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho..."
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev..."
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez..."
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N..."
...,...,...,...
45471,439050,"[Leila Hatami, Kourosh Tahami, Elham Korda]","[Hamid Nematollah, Hamid Nematollah, Farshad M..."
45472,111109,"[Angel Aquino, Perry Dizon, Hazel Orencio, Joe...","[Lav Diaz, Lav Diaz, Dante Perez, Lav Diaz, La..."
45473,67758,"[Erika Eleniak, Adam Baldwin, Julie du Page, J...","[Mark L. Lester, C. Courtney Joyner, Jeffrey G..."
45474,227506,"[Iwan Mosschuchin, Nathalie Lissenko, Pavel Pa...","[Yakov Protazanov, Joseph N. Ermolieff]"


Now merging our crew data with movies_content

In [71]:
type(credits_movies['id'][0])

numpy.int64

In [48]:
#movie_content = movie_content[movie_content['id'].isin(credits_movies['id'])]

In [49]:
#movie_content

,adult,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,id,cast,crew


In [72]:
len(movie_content),len(credits_movies)

(9099, 45476)

In [44]:
len(credits_movies)

45476

In [73]:
movie_content = pd.merge(movie_content,credits_movies,how='inner',on='id')

In [74]:
movie_content

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J..."
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho..."
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev..."
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez..."
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9128,False,NaN,8000000,"[{'id': 18, 'name': 'Drama'}]",NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,85.0,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,"[Sidney Poitier, Wendy Crewson, Jay O. Sanders...","[Gregg Champion, Richard Leder]"
9129,False,NaN,1000000,"[{'id': 53, 'name': 'Thriller'}, {'id': 10749,...",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,150.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,"[Akshay Kumar, Ileana D'Cruz, Esha Gupta, Arja...","[Santosh Thundiiayil, Neeraj Pandey, Shital Bh..."
9130,False,NaN,15050000,"[{'id': 12, 'name': 'Adventure'}, {'id': 18, '...",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,155.0,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,"[Hrithik Roshan, Pooja Hegde, Kabir Bedi, Arun...","[April Ferry, A. R. Rahman, Ashutosh Gowariker..."
9131,False,NaN,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,"[Hiroki Hasegawa, Yutaka Takenouchi, Satomi Is...","[Hideaki Anno, Shinji Higuchi, Akira Ifukube, ..."


In [75]:
movie_content.shape

(9133, 26)

Merging Keywords to our dataframe

In [76]:
keywords = pd.read_csv('/content/drive/MyDrive/datasets/Dataset/keywords.csv')
print(keywords.head())
print(len(keywords))

      id                                           keywords
0    862  [{'id': 931, 'name': 'jealousy'}, {'id': 4290,...
1   8844  [{'id': 10090, 'name': 'board game'}, {'id': 1...
2  15602  [{'id': 1495, 'name': 'fishing'}, {'id': 12392...
3  31357  [{'id': 818, 'name': 'based on novel'}, {'id':...
4  11862  [{'id': 1009, 'name': 'baby'}, {'id': 1599, 'n...
46419


In [77]:
keywords['keywords'][0]

"[{'id': 931, 'name': 'jealousy'}, {'id': 4290, 'name': 'toy'}, {'id': 5202, 'name': 'boy'}, {'id': 6054, 'name': 'friendship'}, {'id': 9713, 'name': 'friends'}, {'id': 9823, 'name': 'rivalry'}, {'id': 165503, 'name': 'boy next door'}, {'id': 170722, 'name': 'new toy'}, {'id': 187065, 'name': 'toy comes to life'}]"

In [78]:
keywords_list = []
for content in keywords['keywords']:
  keywords_list.append(convert(content))

In [ ]:
keywords_list[0]

['jealousy',
 'toy',
 'boy',
 'friendship',
 'friends',
 'rivalry',
 'boy next door',
 'new toy',
 'toy comes to life']

In [79]:
keywords['keywords'] = keywords_list

In [80]:
keywords

,id,keywords
0,862,"[jealousy, toy, boy, friendship, friends, riva..."
1,8844,"[board game, disappearance, based on children'..."
2,15602,"[fishing, best friend, duringcreditsstinger, o..."
3,31357,"[based on novel, interracial relationship, sin..."
4,11862,"[baby, midlife crisis, confidence, aging, daug..."
...,...,...
46414,439050,[tragic love]
46415,111109,"[artist, play, pinoy]"
46416,67758,[]
46417,227506,[]


In [81]:
movie_content = pd.merge(movie_content,keywords,how='inner',on='id')

In [82]:
movie_content['belongs_to_collection'].isna().sum()

7531

In [ ]:
#we'll correct movie_content now
#we'll also correct belongs_to_collection

In [83]:
genres_list = []
for genre in movie_content['genres']:
  genres_list.append(convert(genre))

In [84]:
movie_content['genres'] = genres_list

In [85]:
movie_content['belongs_to_collection'][0]

"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg', 'backdrop_path': '/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg'}"

In [86]:
collection = []
for collect in movie_content['belongs_to_collection']:
  if not pd.isnull(collect):##to take non null values since this column is having null values
    temp = ast.literal_eval(collect)
    collection.append(temp['name'])
  else:
    collection.append('')

In [87]:
movie_content['belongs_to_collection'] = collection

In [88]:
movie_content

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,Toy Story Collection,30000000,"[Animation, Comedy, Family]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney...","[John Lasseter, Joss Whedon, Andrew Stanton, J...","[jealousy, toy, boy, friendship, friends, riva..."
1,False,,65000000,"[Adventure, Fantasy, Family]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,...","[Larry J. Franco, Jonathan Hensleigh, James Ho...","[board game, disappearance, based on children'..."
2,False,Grumpy Old Men Collection,0,"[Romance, Comedy]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop...","[Howard Deutch, Mark Steven Johnson, Mark Stev...","[fishing, best friend, duringcreditsstinger, o..."
3,False,,16000000,"[Comedy, Drama, Romance]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,"[Whitney Houston, Angela Bassett, Loretta Devi...","[Forest Whitaker, Ronald Bass, Ronald Bass, Ez...","[based on novel, interracial relationship, sin..."
4,False,Father of the Bride Collection,0,[Comedy],NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,"[Steve Martin, Diane Keaton, Martin Short, Kim...","[Alan Silvestri, Elliot Davis, Nancy Meyers, N...","[baby, midlife crisis, confidence, aging, daug..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,False,,8000000,[Drama],NaN,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,[],Released,NaN,The Last Brickmaker in America,False,7.0,1.0,"[Sidney Poitier, Wendy Crewson, Jay O. Sanders...","[Gregg Champion, Richard Leder]","[friendship, brick making]"
9215,False,,1000000,"[Thriller, Romance]",NaN,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,"[Akshay Kumar, Ileana D'Cruz, Esha Gupta, Arja...","[Santosh Thundiiayil, Neeraj Pandey, Shital Bh...",[bollywood]
9216,False,,15050000,"[Adventure, Drama, History, Romance]",NaN,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,NaN,Mohenjo Daro,False,6.7,26.0,"[Hrithik Roshan, Pooja Hegde, Kabir Bedi, Arun...","[April Ferry, A. R. Rahman, Ashutosh Gowariker...",[bollywood]
9217,False,,15000000,"[Action, Adventure, Drama, Horror, Science Fic...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,"[Hiroki Hasegawa, Yutaka Takenouchi, Satomi Is...","[Hideaki Anno, Shinji Higuchi, Akira Ifukube, ...","[monster, godzilla, giant monster, destruction..."


In [ ]:
# dataset is created
# what we do here is convert all json format to normal format and combined different dataframes in movie_content dataframe

In [89]:
#remove space between names that are specific
#cast, crew, belong to collection
def combine(text):
  sen = ''
  for word in text:
    new_word = word.replace(" ",'')
    sen = sen + new_word + " "
  
  return sen

In [90]:
new_cast = []
for tmp in movie_content['cast']:
  new_cast.append(combine(tmp))

new_crew = []
for tmp in movie_content['crew']:
  new_crew.append(combine(tmp))

In [91]:
movie_content['cast'] = new_cast
movie_content['crew'] = new_crew

In [92]:
temp = []
for tmp in movie_content['belongs_to_collection']:
  temp.append(tmp.replace(" ","")) 

In [ ]:
movie_content['belongs_to_collection'] = temp

In [93]:
#remove all NaN values from dataset
movie_content = movie_content.fillna('')

In [94]:
#keywords combine
temp_keywords = []
for keyword in movie_content['keywords']:
  temp_keywords.append(" ".join(keyword))

In [95]:
movie_content['keywords'] = temp_keywords

In [96]:
temp_genres = []
for tmp in movie_content['genres']:
  temp_genres.append(" ".join(tmp))

movie_content['genres'] = temp_genres

In [97]:
movie_content

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,Toy Story Collection,30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter JossWhedon AndrewStanton JoelCohe...,jealousy toy boy friendship friends rivalry bo...
1,False,,65000000,Adventure Fantasy Family,,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,RobinWilliams JonathanHyde KirstenDunst Bradle...,LarryJ.Franco JonathanHensleigh JamesHorner Jo...,board game disappearance based on children's b...
2,False,Grumpy Old Men Collection,0,Romance Comedy,,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,WalterMatthau JackLemmon Ann-Margret SophiaLor...,HowardDeutch MarkStevenJohnson MarkStevenJohns...,fishing best friend duringcreditsstinger old men
3,False,,16000000,Comedy Drama Romance,,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker RonaldBass RonaldBass EzraSwerd...,based on novel interracial relationship single...
4,False,Father of the Bride Collection,0,Comedy,,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,SteveMartin DianeKeaton MartinShort KimberlyWi...,AlanSilvestri ElliotDavis NancyMeyers NancyMey...,baby midlife crisis confidence aging daughter ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,False,,8000000,Drama,,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,[],Released,,The Last Brickmaker in America,False,7.0,1.0,SidneyPoitier WendyCrewson JayO.Sanders MaryAl...,GreggChampion RichardLeder,friendship brick making
9215,False,,1000000,Thriller Romance,,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,AkshayKumar IleanaD'Cruz EshaGupta ArjanBajwa ...,SantoshThundiiayil NeerajPandey ShitalBhatia S...,bollywood
9216,False,,15050000,Adventure Drama History Romance,,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,,Mohenjo Daro,False,6.7,26.0,HrithikRoshan PoojaHegde KabirBedi ArunodaySin...,AprilFerry A.R.Rahman AshutoshGowariker Ashuto...,bollywood
9217,False,,15000000,Action Adventure Drama Horror Science Fiction,,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,HirokiHasegawa YutakaTakenouchi SatomiIshihara...,HideakiAnno ShinjiHiguchi AkiraIfukube Hideaki...,monster godzilla giant monster destruction kai...


#Simple Algorithm Based Recommendation System

In [ ]:
## It's a simple formaula based calculation to recommend the movie based on high ratings.
## IMDB follows this method

In [98]:
movie_content

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,Toy Story Collection,30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter JossWhedon AndrewStanton JoelCohe...,jealousy toy boy friendship friends rivalry bo...
1,False,,65000000,Adventure Fantasy Family,,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,RobinWilliams JonathanHyde KirstenDunst Bradle...,LarryJ.Franco JonathanHensleigh JamesHorner Jo...,board game disappearance based on children's b...
2,False,Grumpy Old Men Collection,0,Romance Comedy,,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,WalterMatthau JackLemmon Ann-Margret SophiaLor...,HowardDeutch MarkStevenJohnson MarkStevenJohns...,fishing best friend duringcreditsstinger old men
3,False,,16000000,Comedy Drama Romance,,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker RonaldBass RonaldBass EzraSwerd...,based on novel interracial relationship single...
4,False,Father of the Bride Collection,0,Comedy,,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,SteveMartin DianeKeaton MartinShort KimberlyWi...,AlanSilvestri ElliotDavis NancyMeyers NancyMey...,baby midlife crisis confidence aging daughter ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,False,,8000000,Drama,,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,[],Released,,The Last Brickmaker in America,False,7.0,1.0,SidneyPoitier WendyCrewson JayO.Sanders MaryAl...,GreggChampion RichardLeder,friendship brick making
9215,False,,1000000,Thriller Romance,,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,AkshayKumar IleanaD'Cruz EshaGupta ArjanBajwa ...,SantoshThundiiayil NeerajPandey ShitalBhatia S...,bollywood
9216,False,,15050000,Adventure Drama History Romance,,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,,Mohenjo Daro,False,6.7,26.0,HrithikRoshan PoojaHegde KabirBedi ArunodaySin...,AprilFerry A.R.Rahman AshutoshGowariker Ashuto...,bollywood
9217,False,,15000000,Action Adventure Drama Horror Science Fiction,,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,HirokiHasegawa YutakaTakenouchi SatomiIshihara...,HideakiAnno ShinjiHiguchi AkiraIfukube Hideaki...,monster godzilla giant monster destruction kai...


In [99]:
# v/(v+m) * R + m /(v+m) * C = wr
# v - no.of.vote counts
# c- min.no.of votes required(user defined)
# R- average rating of the movie
# m- average rating for all the movies in the count

In [100]:
vote_counts = movie_content[movie_content['vote_count'].notnull()]['vote_count'].astype(int)

In [101]:
vote_average = movie_content[movie_content['vote_average'].notnull()]['vote_average'].astype(int)

In [102]:
C = vote_average.mean()
C

5.910836316303286

In [112]:
m = vote_counts.quantile(0.85)##it's a hyperparameter. you can select your value

In [104]:
m

722.0

In [113]:
qualified = movie_content[(movie_content['vote_count']>=m) & (movie_content['vote_count'].notnull()) & (movie_content['vote_average'].notnull())][['title','vote_average','vote_count']]

In [106]:
qualified

,title,vote_average,vote_count
0,Toy Story,7.7,5415.0
1,Jumanji,6.9,2413.0
5,Heat,7.7,1886.0
9,GoldenEye,6.6,1194.0
15,Casino,7.8,1343.0
...,...,...,...
9200,The Legend of Tarzan,5.5,2488.0
9201,The Purge: Election Year,6.1,1356.0
9202,Mike and Dave Need Wedding Dates,5.9,901.0
9206,Nerve,7.1,2262.0


In [107]:
def weighted_rating(x):
  v = x['vote_count']
  R = x['vote_average']
  return ((v/(v+m)*R) + (m/(v+m)*C))

In [108]:
qualified['wr']  = qualified.apply(weighted_rating, axis = 1)

In [109]:
qualified

,title,vote_average,vote_count,wr
0,Toy Story,7.7,5415.0,7.489510
1,Jumanji,6.9,2413.0,6.672193
5,Heat,7.7,1886.0,7.204687
9,GoldenEye,6.6,1194.0,6.340305
15,Casino,7.8,1343.0,7.139479
...,...,...,...,...
9200,The Legend of Tarzan,5.5,2488.0,5.592406
9201,The Purge: Election Year,6.1,1356.0,6.034275
9202,Mike and Dave Need Wedding Dates,5.9,901.0,5.904821
9206,Nerve,7.1,2262.0,6.812273


In [110]:
qualified = qualified.sort_values('wr', ascending = False)

In [111]:
qualified.head(10)

,title,vote_average,vote_count,wr
284,The Shawshank Redemption,8.5,8358.0,8.294122
699,The Godfather,8.5,6024.0,8.222891
6981,The Dark Knight,8.3,12269.0,8.167218
2390,Fight Club,8.3,9678.0,8.134137
266,Pulp Fiction,8.3,8670.0,8.116336
321,Forrest Gump,8.2,8147.0,8.013646
7648,Inception,8.1,14075.0,7.993183
8613,Interstellar,8.1,11187.0,7.967279
472,Schindler's List,8.3,4436.0,7.965573
8697,Whiplash,8.3,4376.0,7.961637


#Content Based Recommedation System

In [ ]:
##It recommends movie to the user based on the content.content may be any like (based on director,actor,story)

##Preprocessing Dataset

In [114]:
movie_content

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,Toy Story Collection,30000000,Animation Comedy Family,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,,Toy Story,False,7.7,5415.0,TomHanks TimAllen DonRickles JimVarney Wallace...,JohnLasseter JossWhedon AndrewStanton JoelCohe...,jealousy toy boy friendship friends rivalry bo...
1,False,,65000000,Adventure Fantasy Family,,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,RobinWilliams JonathanHyde KirstenDunst Bradle...,LarryJ.Franco JonathanHensleigh JamesHorner Jo...,board game disappearance based on children's b...
2,False,Grumpy Old Men Collection,0,Romance Comedy,,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,WalterMatthau JackLemmon Ann-Margret SophiaLor...,HowardDeutch MarkStevenJohnson MarkStevenJohns...,fishing best friend duringcreditsstinger old men
3,False,,16000000,Comedy Drama Romance,,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,WhitneyHouston AngelaBassett LorettaDevine Lel...,ForestWhitaker RonaldBass RonaldBass EzraSwerd...,based on novel interracial relationship single...
4,False,Father of the Bride Collection,0,Comedy,,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,SteveMartin DianeKeaton MartinShort KimberlyWi...,AlanSilvestri ElliotDavis NancyMeyers NancyMey...,baby midlife crisis confidence aging daughter ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9214,False,,8000000,Drama,,159550,tt0255313,en,The Last Brickmaker in America,A man must cope with the loss of his wife and ...,...,[],Released,,The Last Brickmaker in America,False,7.0,1.0,SidneyPoitier WendyCrewson JayO.Sanders MaryAl...,GreggChampion RichardLeder,friendship brick making
9215,False,,1000000,Thriller Romance,,392572,tt5165344,hi,रुस्तम,"Rustom Pavri, an honourable officer of the Ind...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,Decorated Officer. Devoted Family Man. Defendi...,Rustom,False,7.3,25.0,AkshayKumar IleanaD'Cruz EshaGupta ArjanBajwa ...,SantoshThundiiayil NeerajPandey ShitalBhatia S...,bollywood
9216,False,,15050000,Adventure Drama History Romance,,402672,tt3859980,hi,Mohenjo Daro,"Village lad Sarman is drawn to big, bad Mohenj...",...,"[{'iso_639_1': 'hi', 'name': 'हिन्दी'}]",Released,,Mohenjo Daro,False,6.7,26.0,HrithikRoshan PoojaHegde KabirBedi ArunodaySin...,AprilFerry A.R.Rahman AshutoshGowariker Ashuto...,bollywood
9217,False,,15000000,Action Adventure Drama Horror Science Fiction,,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0,HirokiHasegawa YutakaTakenouchi SatomiIshihara...,HideakiAnno ShinjiHiguchi AkiraIfukube Hideaki...,monster godzilla giant monster destruction kai...


In [115]:
movie_content.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count', 'cast', 'crew', 'keywords'],
      dtype='object')

In [116]:
#now the question arises which dataset to keep which not to keep
movie_content['original_language'].value_counts()

en    8018
fr     286
ja     206
de     120
it      98
es      88
sv      48
cn      47
zh      39
da      37
ko      31
ru      28
pt      25
fa      21
nl      16
hi      14
fi      13
cs      10
tr       9
no       7
th       7
pl       6
he       6
el       5
hu       4
sr       4
id       3
xx       3
bn       3
nb       2
vi       2
ro       2
ar       2
bs       1
sk       1
is       1
bo       1
ps       1
uk       1
af       1
et       1
lo       1
Name: original_language, dtype: int64

In [117]:
#columns we are dropping
#adult
#budget
#homepage
#imdb_id
#original_language	
#original_title
#imdb_id
#popularity
#poster_path
#production_companies
#production_countries
#release_date
#revenue
#runtime
#spoken_languages
#status
#video
#vote_average
#vote_count

In [118]:
movie_content2 = movie_content.drop(['adult', 'budget','homepage','imdb_id','original_language','original_title','popularity','poster_path','production_companies','production_countries','release_date','revenue','runtime','spoken_languages','status','video','vote_average','vote_count'], axis=1)

In [119]:
final_Mcontent = pd.DataFrame()

In [120]:
#combininng all types of content together like  overview tagline keywords 
final_Mcontent['content'] = movie_content2['tagline'] +" " + movie_content2['overview'] + " " + movie_content2['keywords']

In [121]:
final_Mcontent['keywords'] = movie_content2['belongs_to_collection'] + " " + movie_content2['genres'] + " " + movie_content2['title'] + " " + movie_content2['cast'] + " " + movie_content2['crew']

In [122]:
final_Mcontent

,content,keywords
0,"Led by Woody, Andy's toys live happily in his...",Toy Story Collection Animation Comedy Family T...
1,Roll the dice and unleash the excitement! When...,Adventure Fantasy Family Jumanji RobinWilliam...
2,Still Yelling. Still Fighting. Still Ready for...,Grumpy Old Men Collection Romance Comedy Grump...
3,Friends are the people who let you be yourself...,Comedy Drama Romance Waiting to Exhale Whitne...
4,Just When His World Is Back To Normal... He's ...,Father of the Bride Collection Comedy Father o...
...,...,...
9214,A man must cope with the loss of his wife and...,Drama The Last Brickmaker in America SidneyPo...
9215,Decorated Officer. Devoted Family Man. Defendi...,Thriller Romance Rustom AkshayKumar IleanaD'C...
9216,"Village lad Sarman is drawn to big, bad Mohen...",Adventure Drama History Romance Mohenjo Daro ...
9217,A god incarnate. A city doomed. From the mind ...,Action Adventure Drama Horror Science Fiction...


In [123]:
lemmatizer =WordNetLemmatizer()

In [124]:
stop = stopwords.words('english')
#and i the am an 

In [125]:
def preprocessing(text):
  _text = []
  for i in range(len(text)):
    temp = text[i]

    #extra spaces between words
    temp = re.sub(' +',' ',temp)

    #remove all the punctutaion 
    temp = re.sub(r'[^\w\s]','',temp)

    #remove all the digits
    temp = "".join([i for i in temp if not i.isdigit()])

    #lowercase the sentences
    temp = temp.lower()

    #toknisation
    temp = word_tokenize(temp)

    #lemmatization
    temp = [lemmatizer.lemmatize(word) for word in temp]

    #stopwords
    temp = [word for word in temp if word not in stop]

    _text.append(" ".join(temp))
  
  return _text

In [126]:
final_Mcontent['content'] = preprocessing(final_Mcontent['content'])

In [127]:
final_Mcontent

,content,keywords
0,led woody andys toy live happily room andys bi...,Toy Story Collection Animation Comedy Family T...
1,roll dice unleash excitement sibling judy pete...,Adventure Fantasy Family Jumanji RobinWilliam...
2,still yelling still fighting still ready love ...,Grumpy Old Men Collection Romance Comedy Grump...
3,friend people let never let forget cheated mis...,Comedy Drama Romance Waiting to Exhale Whitne...
4,world back normal surprise life george bank ha...,Father of the Bride Collection Comedy Father o...
...,...,...
9214,man must cope loss wife obsolescence job findi...,Drama The Last Brickmaker in America SidneyPo...
9215,decorated officer devoted family man defending...,Thriller Romance Rustom AkshayKumar IleanaD'C...
9216,village lad sarman drawn big bad mohenjo daro ...,Adventure Drama History Romance Mohenjo Daro ...
9217,god incarnate city doomed mind behind evangeli...,Action Adventure Drama Horror Science Fiction...


##Tf-Idf and cosine similiarty

In [ ]:
##TFIDF is for converting words into vectors
## cosine_simalrity is to calculate the relationship b/w them

In [128]:
final_Mcontent

,content,keywords
0,led woody andys toy live happily room andys bi...,Toy Story Collection Animation Comedy Family T...
1,roll dice unleash excitement sibling judy pete...,Adventure Fantasy Family Jumanji RobinWilliam...
2,still yelling still fighting still ready love ...,Grumpy Old Men Collection Romance Comedy Grump...
3,friend people let never let forget cheated mis...,Comedy Drama Romance Waiting to Exhale Whitne...
4,world back normal surprise life george bank ha...,Father of the Bride Collection Comedy Father o...
...,...,...
9214,man must cope loss wife obsolescence job findi...,Drama The Last Brickmaker in America SidneyPo...
9215,decorated officer devoted family man defending...,Thriller Romance Rustom AkshayKumar IleanaD'C...
9216,village lad sarman drawn big bad mohenjo daro ...,Adventure Drama History Romance Mohenjo Daro ...
9217,god incarnate city doomed mind behind evangeli...,Action Adventure Drama Horror Science Fiction...


In [129]:
tf1 = TfidfVectorizer(ngram_range = (1,2))
tfidf_matrix1 = tf1.fit_transform(final_Mcontent['content'])

In [130]:
tf2 = TfidfVectorizer(ngram_range = (1,2))
tfidf_matrix2 = tf2.fit_transform(final_Mcontent['keywords'])

In [131]:
from sklearn.metrics.pairwise import cosine_similarity

In [132]:
cosine_sim1 = cosine_similarity(tfidf_matrix1,tfidf_matrix1)#contnet

In [133]:
cosine_sim2 = cosine_similarity(tfidf_matrix2,tfidf_matrix2)#keywords

In [134]:
titles = {} #-- index- movie
indices = {} # -- movies -indiex
for i, movie in enumerate(movie_content['title']):
  titles[i] = movie
  indices[movie] = i

In [ ]:
cosine_sim1[0],cosine_sim2[0]

(array([1.        , 0.01003842, 0.001492  , ..., 0.        , 0.00266321,
        0.00404116]),
 array([1.00000000e+00, 8.86680534e-04, 5.49823013e-04, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]))

In [ ]:
def get_recommedation(title):
  idx = indices[title]
  sim_scores1 = list(enumerate(cosine_sim1[idx]))##enumerate gives the o/p in tuple.so list is used to convert them
  sim_scores2 = list(enumerate(cosine_sim2[idx]))
  sim_scores = []
  for i in range(len(sim_scores1)):
    temp = (sim_scores1[i][0],(sim_scores1[i][1] + sim_scores2[i][1])/2)
    sim_scores.append(temp)
  
  sim_scores = sorted(sim_scores,key = lambda x:x[1], reverse =True)
  sim_scores = sim_scores[1:10]
  movie_indices = [i[0] for i in sim_scores]
  movies = []
  for ind in movie_indices:
    movies.append(titles[ind])
  
  return movies

In [ ]:
get_recommedation('X-Men: The Last Stand')

['X2',
 'X-Men',
 'X-Men: Days of Future Past',
 'X-Men: Apocalypse',
 'X-Men: First Class',
 'The Wolverine',
 'Fantastic 4: Rise of the Silver Surfer',
 'Captain America: Civil War',
 'X-Men Origins: Wolverine']

In [ ]:
## disadvantage : it recommends movie only based on contents. It may not be a good movie. it may be the worst movie. it doesn't care about that

#Hybrid Recommedation Systems

In [ ]:
## It's a combination of other filtering method
## it eliminates the disadvantages of other filtering methods
## industrial standard
## here we combine both the above method

In [ ]:
def improved_recommendation(title):
  idx = indices[title]
  sim_scores1 = list(enumerate(cosine_sim1[idx]))
  sim_scores2 = list(enumerate(cosine_sim2[idx]))
  sim_scores = []
  for i in range(len(sim_scores1)):
    temp = (sim_scores1[i][0],(sim_scores1[i][1] + sim_scores2[i][1])/2)
    sim_scores.append(temp)
  sim_scores = sorted(sim_scores,key = lambda x:x[1], reverse =True)
  #we have all the movies that are closely related to the given movie 
  sim_scores = sim_scores[1:30]
  movie_indices = [i[0] for i in sim_scores]
  movies = movie_content.iloc[movie_indices][['title','vote_average','vote_count']]

  vote_counts = movies[movies['vote_count'].notnull()]['vote_count'].astype(int)
  vote_average = movies[movies['vote_average'].notnull()]['vote_average'].astype(int)
  C = vote_average.mean()
  m = vote_counts.quantile(0.60)

  qualified = movies[(movies['vote_count']>=m) & (movies['vote_count'].notnull()) & (movies['vote_average'].notnull())][['title','vote_average','vote_count']]
  qualified['vote_count'] = qualified['vote_count'].astype('int')
  qualified['vote_average'] = qualified['vote_average'].astype('int')
  qualified['wr']  = qualified.apply(weighted_rating, axis = 1)
  qualified = qualified.sort_values('wr', ascending = False).head(10)

  return qualified

In [ ]:
improved_recommendation('The Dark Knight')

,title,vote_average,vote_count,wr
7648,Inception,8,14075,7.898062
6623,The Prestige,8,4510,7.711702
48,The Usual Suspects,8,3334,7.628112
8031,The Dark Knight Rises,7,9263,6.921244
6218,Batman Begins,7,7511,6.904485
7583,Kick-Ass,7,4747,6.856212
8033,Sherlock Holmes: A Game of Shadows,7,3971,6.832436
524,Batman,7,2145,6.725715
8419,Man of Steel,6,6462,5.991039
1134,Batman Returns,6,1706,5.973486


In [ ]:
get_recommedation('The Dark Knight')##this based on content based filtering

['The Dark Knight Rises',
 'Batman Begins',
 'Batman Returns',
 'Batman: Under the Red Hood',
 'Batman Forever',
 'Batman',
 'Batman: The Dark Knight Returns, Part 2',
 'Batman v Superman: Dawn of Justice',
 'Batman: The Dark Knight Returns, Part 1']

## **collaborative** **filtering**

In [ ]:
## it combines users of same type based on the ratings they gave for the movies and recommend articles based on them
##(i.e,)recommend article to one user based on what other user of his same type has watched/read

##Creating emebedding

In [161]:
user_rating = pd.read_csv('/content/drive/MyDrive/datasets/Dataset/ratings_small.csv')

In [162]:
user_rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [163]:
user_rating.shape

(100004, 4)

In [ ]:
max(user_rating['rating']) #ratings are given out of 5

5.0

In [ ]:
# we want a list of movies that are present in our movie_content dataset

In [164]:
user_rating.groupby('movieId').count()['rating']

movieId
1         247
2         107
3          59
4          13
5          56
         ... 
161944      1
162376      1
162542      1
162672      1
163949      1
Name: rating, Length: 9066, dtype: int64

In [165]:
final_user_rating = pd.DataFrame()

In [166]:
final_ratings = user_rating[user_rating['movieId'].isin(movie_content['id'])]

In [167]:
final_ratings

,userId,movieId,rating,timestamp
10,1,1371,2.5,1260759135
11,1,1405,1.0,1260759203
13,1,2105,4.0,1260759139
15,1,2193,2.0,1260759198
16,1,2294,2.0,1260759108
...,...,...,...,...
99981,671,4973,4.5,1064245471
99983,671,4995,4.0,1064891537
99992,671,5816,4.0,1065111963
99993,671,5902,3.5,1064245507


In [168]:
final_ratings[final_ratings["rating"].isnull()]

,userId,movieId,rating,timestamp


In [169]:
final_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32131 entries, 10 to 99996
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     32131 non-null  int64  
 1   movieId    32131 non-null  int64  
 2   rating     32131 non-null  float64
 3   timestamp  32131 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 1.2 MB


In [170]:
final_ratings2 = final_ratings.pivot_table(index = 'movieId' , columns = 'userId' , values = 'rating')

In [171]:
final_ratings2

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.0,NaN,5.0,4.0,NaN,NaN,NaN,NaN
11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90603,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95963,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96821,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [172]:
avg=final_ratings.groupby("movieId")["rating"].mean()

In [144]:
avg

movieId
2         3.401869
5         3.267857
6         3.884615
11        3.689024
12        2.861111
            ...   
90603     2.500000
95963     3.500000
96821     3.928571
103731    5.000000
134368    3.700000
Name: rating, Length: 1548, dtype: float64

In [173]:
final_ratings2 = final_ratings2.fillna(avg)
#treated the missing values with avearage rating got for that particular movie

##Finding cosine similairty

In [174]:
final_ratings2

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
2,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,5.000000,3.411765,3.5,3.0,NaN,3.666667,4.0,NaN,3.166667,4.030303
5,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,3.5,3.0,NaN,3.666667,4.0,NaN,3.166667,4.030303
6,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,4.0,3.4,5.0,4.000000,4.0,NaN,3.166667,4.030303
11,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,3.5,3.4,NaN,3.000000,4.0,NaN,3.166667,4.030303
12,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,3.5,3.4,NaN,3.666667,4.0,NaN,3.166667,4.030303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90603,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,3.5,3.4,NaN,3.666667,4.0,NaN,3.166667,4.030303
95963,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,3.5,3.4,NaN,3.666667,4.0,NaN,3.166667,4.030303
96821,NaN,3.401869,NaN,NaN,3.267857,3.884615,NaN,NaN,NaN,NaN,...,3.111111,3.411765,3.5,3.4,NaN,3.666667,4.0,NaN,3.166667,4.030303


In [175]:
final_ratings2 = final_ratings2.fillna(0)

In [176]:
final_ratings2

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
movieId,,,,,,,,,,,,,,,,,,,,,
2,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,5.000000,3.411765,3.5,3.0,0.0,3.666667,4.0,0.0,3.166667,4.030303
5,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,3.5,3.0,0.0,3.666667,4.0,0.0,3.166667,4.030303
6,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,4.0,3.4,5.0,4.000000,4.0,0.0,3.166667,4.030303
11,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,3.5,3.4,0.0,3.000000,4.0,0.0,3.166667,4.030303
12,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,3.5,3.4,0.0,3.666667,4.0,0.0,3.166667,4.030303
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90603,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,3.5,3.4,0.0,3.666667,4.0,0.0,3.166667,4.030303
95963,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,3.5,3.4,0.0,3.666667,4.0,0.0,3.166667,4.030303
96821,0.0,3.401869,0.0,0.0,3.267857,3.884615,0.0,0.0,0.0,0.0,...,3.111111,3.411765,3.5,3.4,0.0,3.666667,4.0,0.0,3.166667,4.030303


In [177]:
cosine_sim = cosine_similarity(final_ratings2)

In [178]:
cosine_sim.shape

(1548, 1548)

In [179]:
cosine_sim[1]

array([0.93414852, 1.        , 0.93441334, ..., 0.97344175, 0.97563821,
       0.97806357])

In [180]:
id_movie = {}
movie_id = {}
for i,title in enumerate(movie_content['title']):
  id_movie[movie_content['id'][i]] = title
  movie_id[title] = movie_content['id'][i]

In [ ]:
id_movie[2]

'Ariel'

In [181]:
def get_recommendations(title):
    idx = np.where(final_ratings2.index == movie_id[title])[0]##np.where gives tuple of arrays.So to retrieve the first (and in this case, the only) element from the tuple of arrays.
                                                              ## we are using [0]. so the o/p will only be the index and not the tuple
    if len(idx)!=0:
      idx = idx[0]
    else:
      print('movie not present')
      
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:10]
    movies = []
    for i in sim_scores:
      idx = i[0]
      id = final_ratings2.index[idx]
      movies.append(id_movie[id])
    return movies

In [ ]:
get_recommendations('The Shawshank Redemption')##this i got when i compute missing values in the rating column with zero

['Natural Born Killers',
 'The Rolling Stones: Gimme Shelter',
 'Three Colors: Blue',
 'Groundhog Day',
 'Sin City: A Dame to Kill For',
 'King Kong',
 'Ladyhawke',
 'Constantine',
 'The Birds']

In [182]:
get_recommendations('The Shawshank Redemption')##when treated nan values in ratings with avg. rating got for that particular movie

['The Rolling Stones: Gimme Shelter',
 'Ghost',
 'Love on the Run',
 'Captain America: The First Avenger',
 '1408',
 "Lorenzo's Oil",
 'The Hunting Party',
 'Titan A.E.',
 'Touch of Evil']